In [1]:
import openpyxl as excel
from openpyxl.utils import get_column_letter as get_col, get_column_interval, dataframe
from typing import List, Dict
from dataclasses import dataclass, field, asdict
import json

In [2]:
competitiors = excel.load_workbook(filename='competitors.xlsx')

In [3]:
competitiors_sheet = competitiors['Copia de LINKS ACTIVOS ']
competitiors_sheet['A2'].value
get_column_interval('A', 'H')

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']

In [4]:

@dataclass
class MeliItem:
    is_ours: bool
    item_url: str
    sku: str

    

In [5]:
def getHyperLink(cell: excel.cell.Cell) -> str:
    link = cell.value if cell.value else ''
    if cell.hyperlink is not None:
        link = cell.hyperlink.target
    link = link if link.startswith('http') else ''
    return link

In [6]:
row_index = 2
items: Dict[str, Dict] = {}
while (a_value := competitiors_sheet[f"A{row_index}"].value) is not None:
    
    if not (item_link := getHyperLink(competitiors_sheet[f"I{row_index}"])):
        print(f"No link found for {row_index}")
        
    is_ours = competitiors_sheet[f"A{row_index}"].value
    is_ours = True if is_ours == "Propio" else False
    meli_item = MeliItem(is_ours, item_link, competitiors_sheet[f"D{row_index}"].value)
    if meli_item.is_ours:
        assert meli_item.sku not in items, f"{meli_item.sku} already exists"
        items[meli_item.sku] = asdict(meli_item)
        items[meli_item.sku]["competitors"] = []
    elif meli_item.sku in items:
        items[meli_item.sku]["competitors"].append(asdict(meli_item))
    else:
        print(f"WARNING: competitor item came up before our item {meli_item.sku} on row {row_index}")
    row_index += 1

No link found for 32
No link found for 48
No link found for 61
No link found for 67
No link found for 225
No link found for 422
No link found for 513
No link found for 535
No link found for 561
No link found for 564
No link found for 619
No link found for 726


In [7]:
with open('shopsi_data.json', 'w') as f:
    json.dump(items, f, indent=4)